
# 🗃️ Brazilian ecommerce olist

Este é um conjunto de dados públicos de comércio eletrônico brasileiro das compras feitas na loja Olist. O conjunto de dados contém informações de 100 mil pedidos de 2016 a 2018 feitos em vários marketplaces no Brasil. Suas características permitem visualizar um pedido em várias dimensões: desde o status do pedido, preço, pagamento e desempenho de frete até a localização do cliente, atributos do produto e, finalmente, avaliações escritas pelos clientes. Também disponibilizamos um conjunto de dados de geolocalização que relaciona os códigos postais brasileiros às coordenadas lat/long.

Estes são dados comerciais reais, foram anonimizados, e as referências às empresas e parceiros no texto de revisão foram substituídas pelos nomes das grandes casas de Game of Thrones.

In [0]:
%run ../_utils

In [0]:
%run ../_create_database


## dependency

In [0]:
!pip install opendatasets


## download dataset from kaggle

O dataset escolhido foi o [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerces)

Como é um dataset estático (ou quase 100%), não faz sentido adicionar upsert e tampouco streaming.

Mas aqui poderíamos utilizar do AutoLoader, ou até mesmo de alguma ferramenta com CDC, como airbyte

Nossa staging não precisaria existir (apenas caso fossem dados vindos por airbyte, por exemplo). Mas vou criar aqui apenas para exemplificar, pois irei salvar a staging em parquet. Depois disso, todas as camadas serão em Delta

In [0]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce", data_dir="/FileStore/datasets/") # foi para file:/


os arquivos foram salvos em `/FileStore/dataset/brazilian-ecommerce/olist_*.csv``


## Carregando o arquivo .csv para spark dataframe

aqui o .od fez eu usar o workspace files. [mais detalhes aqui](https://docs.databricks.com/en/files/index.html)

In [0]:
df = (
    spark.read.format("csv")
    .options(header="true", inferSchema=True)
    .load("file:/FileStore/datasets/brazilian-ecommerce/olist_customers_dataset.csv")
)


## 🔎 First exploratory data analysis (mini descriptive)

In [0]:
display(df)

In [0]:
print(f"Total de linhas: {df.count()}")


Data set central. Deve ser usado com o uma especie de "tabela fato"
### schema
image

In [0]:
df.printSchema()


## Saving data

salvando todos os .csv no dbf em arquivo parquet

Nossa staging poderia ser um S3 (aws) ou blob storage, mas vai ser diretamente no dbfs mesmo

In [0]:
files = [
    "olist_customers_dataset",
    "olist_orders_dataset",
    "olist_geolocation_dataset",
    "olist_products_dataset",
    "olist_order_items_dataset",
    "olist_sellers_dataset",
    "olist_order_payments_dataset",
    "product_category_name_translation",
    "olist_order_reviews_dataset",
]

In [0]:
for file_name in files:
    # multiline é preciso por conta principalmente dos reviews. pois tem  \n
    df = (
        spark.read.format("csv")
        .options(header="true", inferSchema=True, multiLine=True, quote='"', escape='"')
        .load(f"file:/FileStore/datasets/brazilian-ecommerce/{file_name}.csv")
    )

    save_dataframe(
        df,
        format_mode="parquet",
        target_location=f"dbfs:/FileStore/parquet/brazilian_ecommerce/{file_name}",
    )


Aqui utilizamos de algumas options para leitura do csv

```
.option("quote", "\"") # para que o delimitador de string seja "
.option("header", True) # para que a primeira linha do csv seja lida como header
.option("inferSchema", True) # para que infira o schema automaticamente
.option("multiLine", True) # para casos onde tem quebra de linha no texto
.option("escape", "\"") # para ignorar aspas duplas dentro de uma string (principalmente nos reviews)

```

In [0]:
%fs ls '/FileStore/parquet/brazilian_ecommerce/'

In [0]:
dbutils.notebook.exit("OK")


#### test

In [0]:
display(spark.read.parquet('/FileStore/parquet/brazilian_ecommerce/olist_order_reviews_dataset/'))